<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#libraries-and-widgets" data-toc-modified-id="libraries-and-widgets-0.0.0.1"><span class="toc-item-num">0.0.0.1&nbsp;&nbsp;</span>libraries and widgets</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Neuromatch-Academy-Project---Steinmetz-Dataset---Mouse-Mind-Reading" data-toc-modified-id="Neuromatch-Academy-Project---Steinmetz-Dataset---Mouse-Mind-Reading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Neuromatch Academy Project - Steinmetz Dataset - Mouse Mind Reading</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#explore-dictionary-for-session" data-toc-modified-id="explore-dictionary-for-session-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>explore dictionary for session</a></span></li></ul></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></span><ul class="toc-item"><li><span><a href="#via-google-drive-directly" data-toc-modified-id="via-google-drive-directly-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>via google drive directly</a></span></li><li><span><a href="#cleanup-channel-infos" data-toc-modified-id="cleanup-channel-infos-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>cleanup channel infos</a></span></li><li><span><a href="#cleaning-up-spikes" data-toc-modified-id="cleaning-up-spikes-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>cleaning up spikes</a></span></li><li><span><a href="#cleanup-trial-info" data-toc-modified-id="cleanup-trial-info-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>cleanup trial info</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt
import qgrid

from nma_class import NMA_project

%matplotlib inline

#### libraries and widgets
- qgrid
- nbextentions

In [2]:
# only if you are on jupyter notebook not if on colab
import qgrid

In [ ]:
# only if you are on google coalb not if you are on jupyter notebook
from google.colab import drive
drive.mount('/content/gdrive')

# Neuromatch Academy Project - Steinmetz Dataset - Mouse Mind Reading

In [2]:
# link to parent folder with all datasets
#main_folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#main_folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#main_folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
main_folder = "E:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"

# create session object
nma_obj = NMA_project(main_folder) 

# default session is initialized, extract interesting dataframes in dedicated variables
session = nma_obj.std_session



In [5]:
#channels_df = session['channels_df']
spikes_df = session['spikes_df']
clusters_df = session['clusters_df']
#trials_df = session['trials_df']

In [5]:
qgrid.show_grid(clusters_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [3]:
folder = "E:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset/Cori_2016-12-14"

# good cells and brain regions
good_cells, brain_region, br, phy_label = nma_obj.get_good_cells(folder)
# get all spikes
spk_spikes, spk_clusters = nma_obj.get_all_spikes(folder)
# event types
response, vis_right, vis_left, feedback_type, included_tr, rep_nrs = nma_obj.get_event_types(folder)
# event timing
response_times, visual_times, gocue_times, feedback_times, interval_times, rsp, feedback, gocue, interval = nma_obj.get_event_times(folder)   
# get passive trials
vis_times_p, vis_right_p, vis_left_p, gocue_p = nma_obj.get_passive(folder)
visual_times = np.vstack((visual_times, vis_times_p))
vis_right = np.hstack((vis_right, vis_right_p))
vis_left  = np.hstack((vis_left, vis_left_p))
# get spikes and clusters of spikes
spikes, sclust    = nma_obj.get_spikes(folder)
# wheel traces
wheel, wheel_times = nma_obj.get_wheel(folder)
# load the pupil
pup, xy, pup_times = nma_obj.get_pup(folder)
# load the LFP
#L, ba_lfp = nma_obj.get_LFP(folder, br, visual_times-T0, dT, dt, T0)
# trials loader
S  = nma_obj.psth(spikes, sclust,   visual_times-nma_obj.T0, nma_obj.dT, nma_obj.dt)
# wheel trials
W = nma_obj.wpsth(wheel, wheel_times,   visual_times-nma_obj.T0, nma_obj.dT, nma_obj.dt)
# pupil loader
P = nma_obj.ppsth(pup, pup_times,   visual_times-nma_obj.T0, nma_obj.dT, nma_obj.dt)
# add spike waveform information
twav, w, u = nma_obj.get_waves(folder)
good_cells = good_cells * (np.mean(S, axis=(1,2))>0)
S  = S[good_cells].astype('int8') 


session_test = dict()
# _p mens for passif trials
# number of active trials
ntrials = len(response)
session_test['clusters'] = good_cells
session_test['brain_area'] = brain_region[good_cells]
session_test['spk']=S[:, :ntrials, :]
session_test['spks_binned'] = S[:, :ntrials, :]
session_test['wheel'] = W[np.newaxis, :ntrials, :]
session_test['pupil'] = P[:, :ntrials, :]
session_test['response'] = response
session_test['contrast_right'] = vis_right[:ntrials]
session_test['contrast_left'] = vis_left[:ntrials]
session_test['response_time'] = rsp
session_test['feedback_time'] = feedback
session_test['feedback_type'] = feedback_type  
session_test['gocue'] = gocue
#session_test['mouse_name'] = folder.split('\\')[1].split('_')[0]
#session_test['date_exp'] = folder.split('\\')[1].split('_')[1]
session_test['trough_to_peak'] = twav[good_cells].flatten()
session_test['waveform_w'] = w[good_cells].astype('float32')
session_test['waveform_u'] = u[good_cells].astype('float32')
session_test['bin_size'] = nma_obj.dt
session_test['stim_onset'] = nma_obj.T0
session_test['spks_passive'] = S[:, ntrials:, :]
session_test['wheel_passive'] = W[np.newaxis, ntrials:, :]
session_test['pupil_passive'] = P[:, ntrials:, :]
#session_test['lfp_passive'] = L[:, ntrials:, :]
session_test['contrast_right_passive'] = vis_right[ntrials:]
session_test['contrast_left_passive'] = vis_left[ntrials:]

In [6]:
clusters = np.unique(spk_clusters)
clusters_df = pd.DataFrame({'label':phy_label}, index=clusters )
clusters_df['label'] = clusters_df['label'].apply( lambda label: 'good' if label==2 else ('mua' if label==1 else 'bad') )
# drop rows with bad cells
#clusters_df.drop( clusters_df[(clusters_df['label']=='bad')].index,axis=0,inplace=True)
# create spikes colum with spiketimes

spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
# only select good clusters
for group, frame in spikes_df.groupby('cluster'):
    spk['spikes'][group] = frame['spike_times'].values
#merge spike column with clusters_df
clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
# set index name
clusters_df.index.name='cluster'
# number of spikes per cluster
clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )
# add recording position
clusters_df['recording area'] = brain_region

In [11]:
response_times, visual_times, gocue_times, feedback_times, interval_times, rsp, feedback, gocue, interval = nma_obj.get_event_times(folder)   

In [19]:
interval_times

array([[  62.90028371,   67.42348445],
       [  68.42083772,   73.60447565],
       [  74.60290237,   78.00675701],
       [  79.00365324,   84.50677806],
       [  85.50179514,   88.62133641],
       [  89.61759177,   93.17147139],
       [  94.16770229,   98.11970104],
       [  99.1176188 ,  101.90397514],
       [ 102.90225485,  108.66371181],
       [ 109.66196556,  113.31976071],
       [ 114.31820856,  117.83466087],
       [ 118.83384016,  127.4047829 ],
       [ 128.40256754,  131.8530433 ],
       [ 132.84838428,  135.6201119 ],
       [ 136.61738457,  139.12007717],
       [ 140.11648769,  144.73529553],
       [ 145.73070082,  155.9513227 ],
       [ 156.94839312,  164.60097163],
       [ 165.59694565,  169.86748672],
       [ 170.86362616,  173.91689143],
       [ 174.91382177,  179.03884663],
       [ 180.03654192,  186.68360938],
       [ 187.68056025,  191.03249953],
       [ 192.03032523,  196.11566569],
       [ 197.1117191 ,  201.19931665],
       [ 202.19646615,  2

In [23]:
trials_df = pd.DataFrame({'included':included_tr,
            'repetition number':rep_nrs,
            'start time':interval_times[:,0],
            'stim time':visual_times[:ntrials, :].flatten(),
            #'go cue':gocue_times,
            'response time':response_times.flatten(),
            'feedback time':feedback_times.flatten(),
            'end time':interval_times[:,1],
            'stim contrast left':vis_left[:ntrials].flatten(),
            'stim contrast right':vis_right[:ntrials].flatten(),
            'response choice':response,
            'feedback type':feedback_type,
            })
trials_df = trials_df.astype(dtype= {"repetition number":"int8", 
                                     'stim contrast left':'int8', 
                                     'stim contrast right':'int8',
                                     'response choice': 'int8',
                                     'feedback type':'int8'
                                    })

,included,repetition number,start time,stim time,response time,feedback time,end time,stim contrast left,stim contrast right,response choice,feedback type
0,True,1,62.900284,65.269408,66.419612,66.456227,67.423484,1,0,1,1
1,True,1,68.420838,71.202703,72.602206,72.640326,73.604476,0,0,-1,1
2,True,1,74.602902,76.052380,77.001671,77.038396,78.006757,1,0,1,1
3,True,1,79.003653,81.235263,83.502065,83.531699,84.506778,0,0,0,1
4,True,1,85.501795,86.800952,87.617727,87.628565,88.621336,0,1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...
209,False,5,1109.529226,1110.110867,1111.564004,1111.571290,1112.569226,0,1,1,-1
210,False,6,1113.562081,1114.277333,1115.497692,1115.508153,1116.500302,0,1,1,-1
211,False,7,1117.495128,1117.926592,1120.232805,1120.237428,1121.238999,0,1,0,-1
212,False,8,1122.231454,1122.576266,1124.854126,1124.857902,1125.861004,0,1,0,-1


In [11]:
feedback_type.shape

(214,)

In [24]:
# add wheelmovement to trials dv ===============
#wheel = W[np.newaxis, :, :]
whl = pd.DataFrame( {'wheel movement':np.zeros(ntrials)}, dtype=object) 
for row in range(W.shape[0]):
    whl['wheel movement'][row] = W[row,:]

#merge trials with wheel
trials_df = pd.merge(trials_df, whl, how='right', left_index=True, right_index=True)

# add trials df to session
#session['trials_df'] = trials_df

In [25]:
trials_df

,included,repetition number,start time,stim time,response time,feedback time,end time,stim contrast left,stim contrast right,response choice,feedback type,wheel movement
0,True,1,62.900284,65.269408,66.419612,66.456227,67.423484,1,0,1,1,"[0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0,..."
1,True,1,68.420838,71.202703,72.602206,72.640326,73.604476,0,0,-1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,True,1,74.602902,76.052380,77.001671,77.038396,78.006757,1,0,1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,True,1,79.003653,81.235263,83.502065,83.531699,84.506778,0,0,0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,True,1,85.501795,86.800952,87.617727,87.628565,88.621336,0,1,1,-1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
209,False,5,1109.529226,1110.110867,1111.564004,1111.571290,1112.569226,0,1,1,-1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
210,False,6,1113.562081,1114.277333,1115.497692,1115.508153,1116.500302,0,1,1,-1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
211,False,7,1117.495128,1117.926592,1120.232805,1120.237428,1121.238999,0,1,0,-1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
212,False,8,1122.231454,1122.576266,1124.854126,1124.857902,1125.861004,0,1,0,-1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


#### explore dictionary for session
- each ession is a dictionary
- each element kontains eather a numpy array or a dataframe
- the most important dataframes we are going to work with are:
    - clusters_df: all the spikes and clusters
    - trials_df: all the behaviour data for all trials

In [4]:
for key, value in nma_obj.std_session.items():
    print(key)

clusters
brain_area
spk
spks_binned
wheel
pupil
response
contrast_right
contrast_left
response_time
feedback_time
feedback_type
gocue
mouse_name
date_exp
trough_to_peak
waveform_w
waveform_u
bin_size
stim_onset
spks_passive
wheel_passive
pupil_passive
contrast_right_passive
contrast_left_passive
spikes_df


In [3]:
qgrid.show_grid(nma_obj.std_session['spikes_df'])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Load Data


### via google drive directly

In [3]:
def load_session(folder):
    """
    load all files from given folder to dict
    return dict with key as filenames
    """
    # load session files ===========================================
    from os import listdir
    from os.path import isfile, join
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    names = ['_'.join(f.split('.')[:-1]) for f in files]
    session = dict()
    for (file_, name_) in zip(files, names):
        if file_.split('.')[-1] == 'npy':
        session[name_] = np.load(folder+'/'+file_)
        if file_.split('.')[-1] == 'tsv':
        session[name_] = pd.read_table(folder+'/'+file_)

    # pars session info two dataframe ===============================
    # create spike_df tataframe, with each spike time and the cluster it belongs to
    spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 

    # create cluster dataframe information and spikes for each cluster
    clusters = np.unique(session['spikes_clusters'])
    # create number of spikes, phy2 manual cluster, 
    # parse phy annotation
    clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
    # pars labels in strings 1 = good 3 = mua
    #clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )

    # create spikes colum with spiketimes
    spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
    for group, frame in spikes_df.groupby('cluster'):
        spk['spikes'][group] = frame['spike_times'].values

    #merge spike column with clusters_df
    clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
    # set index name
    clusters_df.index.name='cluster'
    # number of spikes per cluster
    clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

# def load_session(folder, sessions):
#   """
#   load all files in each given session and add to session dict
#   returs a dict of sessions, with each session a dict of files
#   """
#   sessions_data = dict()
#   for session in sessions:
#     current_session = dict()
#     sessions[session] = 

In [7]:
# example google colab load session
#folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
#folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"

subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
sessions = [folder.split('/')[-1] for folder in subfolders]

# load first session
session = load_session(subfolders[0])

KeyboardInterrupt: 

In [10]:
folder = "Z:/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset/Cori_2016-12-14"
session = load_session(folder)

In [11]:
for key in session.keys():
  print(key)

Cori_2016-12-14_M2_g0_t0_imec_lf_timestamps
Cori_2016-12-14_V1_g0_t0_imec_lf_timestamps
channels_brainLocation
channels_probe
channels_rawRow
channels_site
channels_sitePositions
clusters__phy_annotation
clusters_depths
clusters_originalIDs
clusters_peakChannel
clusters_probes
clusters_templateWaveformChans
clusters_templateWaveforms
clusters_waveformDuration
eye_area
eye_timestamps
eye_xyPos
face_motionEnergy
face_timestamps
lickPiezo_raw
lickPiezo_timestamps
licks_times
passiveBeeps_times
passiveValveClick_times
passiveVisual_contrastLeft
passiveVisual_contrastRight
passiveVisual_times
passiveWhiteNoise_times
probes_description
probes_insertion
probes_rawFilename
probes_sitePositions
sparseNoise_positions
sparseNoise_times
spikes_amps
spikes_clusters
spikes_depths
spikes_times
spontaneous_intervals
trials_feedbackType
trials_feedback_times
trials_goCue_times
trials_included
trials_intervals
trials_repNum
trials_response_choice
trials_response_times
trials_visualStim_contrastLeft
tria

### cleanup channel infos

In [76]:
# load channel brain location infos
brain = pd.DataFrame(session['channels_brainLocation'])
# load channel probes
site = pd.DataFrame(session['channels_sitePositions'], columns=['channel 0', 'channel 1'])
# merge with channles_df
channels_df = pd.merge(brain, site, how='inner', left_index=True, right_index=True)
# load probe, row and site
meta = pd.DataFrame({'probe':session['channels_probe'][:,0],
                            'raw row':session['channels_rawRow'][:,0],
                            'channels_site':session['channels_site'][:,0] })
# merge metha with channel_df
channels_df = pd.merge(channels_df, meta, how='inner', left_index=True, right_index=True)

In [77]:
qgrid.show_grid(channels_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### cleaning up spikes

In [12]:
# create spike_df tataframe, with each spike time and the cluster it belongs to
spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 
qgrid.show_grid(spikes_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [55]:
# create cluster dataframe information and spikes for each cluster
clusters = np.unique(session['spikes_clusters'])
# create number of spikes, phy2 manual cluster, 
# parse phy annotation
clusters_df = pd.DataFrame({'lable':session['clusters__phy_annotation'][:,0], 'probe':session['clusters_probes'][:,0], 'peak channel':session['clusters_peakChannel'][:,0]}, index=clusters )
# pars labels in strings 1 = good 3 = mua
#clusters_df['lable'] = clusters_df['lable'].apply( lambda lable: 'good' if lable==3 else ('mua' if lable==1 else 'bad') )



# create spikes colum with spiketimes
spk = pd.DataFrame( {'spikes':np.zeros(len(clusters), dtype=object)}, index=clusters )
for group, frame in spikes_df.groupby('cluster'):
    spk['spikes'][group] = frame['spike_times'].values

#merge spike column with clusters_df
clusters_df = pd.merge(clusters_df, spk, how='right', left_index=True, right_index=True)
# set index name
clusters_df.index.name='cluster'
# number of spikes per cluster
clusters_df['total spikes'] = clusters_df['spikes'].apply(lambda row: len(row) )

In [56]:
qgrid.show_grid(clusters_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### cleanup trial info

In [11]:
print(session['trials_feedbackType'].shape)
print(session['trials_feedback_times'].shape)
print(session['trials_goCue_times'].shape)
print(session['trials_included'].shape)
print(session['trials_intervals'].shape)
print(session['trials_repNum'].shape)
print(session['trials_response_choice'].shape)
print(session['trials_response_times'].shape)
print(session['trials_visualStim_contrastLeft'].shape)
print(session['trials_visualStim_contrastRight'].shape)
print(session['trials_visualStim_times'].shape)

(214, 1)
(214, 1)
(214, 1)
(214, 1)
(214, 2)
(214, 1)
(214, 1)
(214, 1)
(214, 1)
(214, 1)
(214, 1)
(214, 1)


In [20]:
# create trials dataframe
trials_df = pd.DataFrame({'included':session['trials_included'][:,0],
                          'repetition number':session['trials_repNum'][:,0],
                          # times of events
                          'go cue':session['trials_goCue_times'][:,0],
                          'stim time':session['trials_visualStim_times'][:,0],
                          'response time':session['trials_response_times'][:,0],
                          'feedback time':session['trials_feedback_times'][:,0],
                          
                          #'intervals':session['trials_intervals'][:,0], # same as go cue
                          
                          
                          'go cue':session['trials_intervals'][:,0],
                          
                          'stim contrast left':session['trials_visualStim_contrastLeft'][:,0],
                          'stim contrast right':session['trials_visualStim_contrastRight'][:,0],
                          
                          'response choice':session['trials_response_choice'][:,0],
                          'response time':session['trials_response_times'][:,0],
                          'feedback type':session['trials_feedbackType'][:,0],
                          
                         })

In [21]:
qgrid.show_grid(trials_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [25]:
# add wheelmovement
print(session['wheel_position'].shape)
print(session['wheel_timestamps'].shape)
print(session['wheelMoves_intervals'].shape)
print(session['wheelMoves_type'].shape)

(6570000, 1)
(2, 2)
(507, 2)
(507, 1)


In [30]:
wheel_df = pd.merge( pd.DataFrame(session['wheelMoves_intervals']), pd.DataFrame(session['wheelMoves_type']), how='left', left_index=True, right_index=True)

In [31]:
qgrid.show_grid(pd.DataFrame(wheel_df))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…